In [60]:
%matplotlib inline
import matplotlib.pyplot as plt


import numpy as np
import random
import time
import os
import gc
import math

from keras.models import Sequential, clone_model
from keras.layers import Dense, Flatten, Conv2D, InputLayer, BatchNormalization
from keras.callbacks import CSVLogger, TensorBoard
from keras.optimizers import Adam
import keras.backend as K
import rl
#from rl.layers import NoisyNetDense
from rl.agents.dqn import DQNAgent
from rl.policy import GreedyQPolicy
from keras import initializers
import keras.engine
from keras.engine import InputSpec
from keras.callbacks import LearningRateScheduler
import collections, itertools

import gym

plt.rcParams['figure.figsize'] = (9, 9)

In [61]:
class NoisyDense(Dense):
    def __init__(self, units, **kwargs):
        self.output_dim = units
        super(NoisyDense, self).__init__(units, **kwargs)

    def build(self, input_shape):
        assert len(input_shape) >= 2
        self.input_dim = input_shape[-1]

        self.kernel = self.add_weight(shape=(self.input_dim, self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=None,
                                      constraint=None)

        # Zweiter Kernel (trainable weights) für Steuerung des Zufalls.
        self.kernel_sigma = self.add_weight(shape=(self.input_dim, self.units),
                                      initializer=initializers.Constant(0.017),
                                      name='sigma_kernel',
                                      regularizer=None,
                                      constraint=None)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=None,
                                        constraint=None)

            # trainable, Steuerung des Zufalls des Bias.
            self.bias_sigma = self.add_weight(shape=(self.units,),
                                        initializer=initializers.Constant(0.017),
                                        name='bias_sigma',
                                        regularizer=None,
                                        constraint=None)
        else:
            self.bias = None

        self.input_spec = InputSpec(min_ndim=2, axes={-1: self.input_dim})
        self.built = True

    def call(self, inputs):
        # Erzeugen der Matrix mit Zufallszahlen (bei jedem Aufruf neu erzeugt) - Vektor-Version
        # (siehe Noisy Nets Paper) wäre effizienter.
        self.kernel_epsilon = K.random_normal(shape=(self.input_dim, self.units))

        w = self.kernel + K.tf.multiply(self.kernel_sigma, self.kernel_epsilon)
        output = K.dot(inputs, w)

        if self.use_bias:
            # Erzeugung Zufallsvektor für Bias-Zufall.
            self.bias_epsilon = K.random_normal(shape=(self.units,))

            b = self.bias + K.tf.multiply(self.bias_sigma, self.bias_epsilon)
            output = output + b
        if self.activation is not None:
            output = self.activation(output)
        return output

### Create environment

In [347]:
env = gym.make("MsPacman-ram-v0")
obs = env.reset()
done = False
score = 0
env.action_space  # actions are integers from 0 to 8

Discrete(9)

### Create DQN model

In [348]:
def create_dqn_model(input_shape, nb_actions, dense_layers, dense_units):
    model = Sequential()
    model.add(InputLayer(input_shape=input_shape))
    model.add(NoisyDense(units=dense_units, activation='relu'))
    #model.add(NoisyDense(units=dense_units, activation='relu'))
    for i in range(dense_layers):
        #model.add(NoisyDense(units=dense_units, activation='relu'))
        model.add(Dense(units=dense_units, activation='relu'))
    model.add(NoisyDense(units=dense_units, activation='relu'))
    for i in range(dense_layers):
        #model.add(NoisyDense(units=dense_units, activation='relu'))
        model.add(Dense(units=dense_units, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dense(nb_actions, activation='linear'))
    return model

In [349]:
input_shape = obs.shape
nb_actions = env.action_space.n  # 9
dense_layers = 5
dense_units = 256

online_network = create_dqn_model(input_shape, nb_actions, dense_layers, dense_units)
online_network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 128)               0         
_________________________________________________________________
noisy_dense_48 (NoisyDense)  (None, 256)               66048     
_________________________________________________________________
dense_257 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_258 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_259 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_260 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_261 (Dense)            (None, 256)               65792     
__________

Add epsilon greedy exploration strategy

In [350]:
def epsilon_greedy(q_values, epsilon, n_outputs):
    if random.random() < epsilon:
        return random.randrange(n_outputs)  # random action
    else:
        return np.argmax(q_values)          # q-optimal action

Online network stores explored information in a *replay memory*, a double-ended queue (deque).

In [351]:
from collections import deque

replay_memory_maxlen = 1000000
replay_memory = deque([], maxlen=replay_memory_maxlen)

Write weights from online model into target model

In [352]:
target_network = clone_model(online_network)
target_network.set_weights(online_network.get_weights())

### Training Double DQN model

In [353]:
name = 'MsPacman_DQN'  # used in naming files (weights, logs, etc)
n_steps = 50000        # total number of training steps (= n_epochs)
warmup = 1000         # start training after warmup iterations
training_interval = 20  # period (in actions) between training steps
save_steps = int(n_steps/10)  # period (in training steps) between storing weights to file
copy_steps = 500      # period (in training steps) between updating target_network weights
gamma = 0.8            # discount rate
skip_start = 90        # skip the start of every game (it's just freezing time before game starts)
batch_size = 128        # size of minibatch that is taken randomly from replay memory every training step
double_dqn = True     # whether to use Double-DQN approach or simple DQN (see above)
# eps-greedy parameters: we slowly decrease epsilon from eps_max to eps_min in eps_decay_steps
eps_max = 1.0
eps_min = 0.05
eps_decay_steps = int(n_steps/2)
learning_rate = 0.001

Mean squared error loss function

In [354]:
def mean_q(y_true, y_pred):
    return K.mean(K.max(y_pred, axis=-1))


online_network.compile(optimizer=Adam(lr=learning_rate), loss='mse', metrics=[mean_q])

Folder for trained weights

In [355]:
if not os.path.exists(name):
    os.makedirs(name)
    
weights_folder = os.path.join(name, 'weights_2')
if not os.path.exists(weights_folder):
    os.makedirs(weights_folder)

In [356]:
csv_logger = CSVLogger(os.path.join(name, 'log.csv'), append=True, separator=';')

Next chunk of code explores the game, trains online network and periodically copies weights to target network as explained above.

In [186]:
#obs

In [357]:
# counters:
step = 0          # training step counter (= epoch counter)
iteration = 0     # frames counter
episodes = 0      # game episodes counter
done = True       # indicator that env needs to be reset

episode_scores = []  # collect total scores in this list and log it later

while step < n_steps:
    if done:  # game over, restart it
        obs = env.reset()
        score = 0  # reset score for current episode
        for skip in range(skip_start):  # skip the start of each game (it's just freezing time before game starts)
            obs, reward, done, info = env.step(0)
            score += reward
        state = obs
        episodes += 1
    
    # Online network evaluates what to do
    iteration += 1
    q_values = online_network.predict(np.array([state]))[0]  # calculate q-values using online network
    # select epsilon (which linearly decreases over training steps):
    epsilon = max(eps_min, eps_max - (eps_max-eps_min) * step/eps_decay_steps)
    action = epsilon_greedy(q_values, epsilon, nb_actions)
    # Play:
    obs, reward, done, info = env.step(action)
    score += reward
    if done:
        episode_scores.append(score)
    next_state = obs
    # Let's memorize what just happened
    replay_memory.append((state, action, reward, next_state, done))
    state = next_state

    if iteration >= warmup and iteration % training_interval == 0:
        # learning branch
        step += 1
        #index = random.randint(5*batch_size,len(replay_memory))
        #minibatch = replay_memory[(index-5*batch_size):index]
        #list(itertools.islice(q, 3, 7))
        #minibatch = random.sample(replay_memory, batch_size)
        
        minibatch = []
        selectcnt = 4
        batch_counter = 0
        batch_limit = batch_size/selectcnt
        batch_selection_complete = False
        batch_indices = []
        
        while(not batch_selection_complete):
            cstart = random.randint(0, len(replay_memory)-selectcnt)
            
            if(any(ind in batch_indices for ind in (cstart, cstart+1, cstart+2, cstart+3))):
                continue
                
            batch_indices.append(cstart)
            
            minibatch = (minibatch + (list(itertools.islice(replay_memory, cstart, cstart+selectcnt))))
            
            batch_counter += 1
            
            if(batch_counter == batch_limit):
                batch_selection_complete = True
        
        replay_state = np.array([x[0] for x in minibatch])
        replay_action = np.array([x[1] for x in minibatch])
        replay_rewards = np.array([x[2] for x in minibatch])
        replay_next_state = np.array([x[3] for x in minibatch])
        replay_done = np.array([x[4] for x in minibatch], dtype=int)

        # calculate targets (see above for details)
        if double_dqn == False:
            # DQN
            target_for_action = replay_rewards + (1-replay_done) * gamma * \
                                    np.amax(target_network.predict(replay_next_state), axis=1)
        else:
            # Double DQN
            best_actions = np.argmax(online_network.predict(replay_next_state), axis=1)
            target_for_action = replay_rewards + (1-replay_done) * gamma * \
                                    target_network.predict(replay_next_state)[np.arange(batch_size), best_actions]

        target = online_network.predict(replay_state)  # targets coincide with predictions ...
        target[np.arange(batch_size), replay_action] = target_for_action  #...except for targets with actions from replay
        
        # Train online network
        online_network.fit(replay_state, target, epochs=step, verbose=2, initial_epoch=step-1, callbacks=[csv_logger])

        # Periodically copy online network weights to target network
        if step % copy_steps == 0:
            target_network.set_weights(online_network.get_weights())
        # And save weights
        if step % save_steps == 0:
            online_network.save_weights(os.path.join(weights_folder, 'weights_{}.h5f'.format(step)))
            gc.collect()  # also clean the garbage

Epoch 1/1
 - 4s - loss: 7.8264 - mean_q: 5.3473
Epoch 2/2
 - 0s - loss: 1.4944 - mean_q: 3.3109
Epoch 3/3
 - 0s - loss: 2.0898 - mean_q: 2.9269
Epoch 4/4
 - 0s - loss: 1.0053 - mean_q: 2.7818
Epoch 5/5
 - 0s - loss: 0.7401 - mean_q: 2.8516
Epoch 6/6
 - 0s - loss: 0.6718 - mean_q: 2.0931
Epoch 7/7
 - 0s - loss: 1.0269 - mean_q: 2.1961
Epoch 8/8
 - 0s - loss: 0.9665 - mean_q: 2.2626
Epoch 9/9
 - 0s - loss: 0.3336 - mean_q: 1.9475
Epoch 10/10
 - 0s - loss: 0.9297 - mean_q: 1.7533
Epoch 11/11
 - 0s - loss: 0.4528 - mean_q: 1.5813
Epoch 12/12
 - 0s - loss: 0.6782 - mean_q: 1.8618
Epoch 13/13
 - 0s - loss: 0.7471 - mean_q: 1.9326
Epoch 14/14
 - 0s - loss: 0.4878 - mean_q: 1.5536
Epoch 15/15
 - 0s - loss: 0.4208 - mean_q: 1.6794
Epoch 16/16
 - 0s - loss: 0.9099 - mean_q: 1.2373
Epoch 17/17
 - 0s - loss: 0.3558 - mean_q: 1.3573
Epoch 18/18
 - 0s - loss: 0.9398 - mean_q: 1.1987
Epoch 19/19
 - 0s - loss: 0.5923 - mean_q: 1.5583
Epoch 20/20
 - 0s - loss: 0.5811 - mean_q: 1.4080
Epoch 21/21
 - 0s 

Epoch 160/160
 - 0s - loss: 0.2976 - mean_q: -1.4256e-01
Epoch 161/161
 - 0s - loss: 0.3003 - mean_q: -1.4051e-01
Epoch 162/162
 - 0s - loss: 0.3936 - mean_q: -1.5785e-01
Epoch 163/163
 - 0s - loss: 0.3962 - mean_q: -2.5465e-01
Epoch 164/164
 - 0s - loss: 0.0416 - mean_q: -1.8592e-01
Epoch 165/165
 - 0s - loss: 0.6482 - mean_q: -3.2683e-01
Epoch 166/166
 - 0s - loss: 0.4929 - mean_q: -1.3284e-01
Epoch 167/167
 - 0s - loss: 0.3275 - mean_q: -1.4314e-01
Epoch 168/168
 - 0s - loss: 0.5752 - mean_q: -1.6488e-01
Epoch 169/169
 - 0s - loss: 0.5256 - mean_q: -3.7947e-01
Epoch 170/170
 - 0s - loss: 0.8349 - mean_q: -1.8146e-01
Epoch 171/171
 - 0s - loss: 0.3308 - mean_q: -7.2987e-02
Epoch 172/172
 - 0s - loss: 0.6893 - mean_q: -4.8802e-02
Epoch 173/173
 - 0s - loss: 0.6145 - mean_q: -3.2514e-02
Epoch 174/174
 - 0s - loss: 0.5226 - mean_q: -1.4682e-02
Epoch 175/175
 - 0s - loss: 1.0706 - mean_q: -2.4137e-03
Epoch 176/176
 - 0s - loss: 0.6394 - mean_q: 0.0220
Epoch 177/177
 - 0s - loss: 0.6436 -

Epoch 314/314
 - 0s - loss: 0.9895 - mean_q: 0.0982
Epoch 315/315
 - 0s - loss: 0.8113 - mean_q: 0.1236
Epoch 316/316
 - 0s - loss: 0.4929 - mean_q: 0.1517
Epoch 317/317
 - 0s - loss: 0.4553 - mean_q: 0.1810
Epoch 318/318
 - 0s - loss: 0.5060 - mean_q: 0.2013
Epoch 319/319
 - 0s - loss: 1.0794 - mean_q: 0.2151
Epoch 320/320
 - 0s - loss: 0.1802 - mean_q: 0.2457
Epoch 321/321
 - 0s - loss: 0.6376 - mean_q: 0.2921
Epoch 322/322
 - 0s - loss: 0.9582 - mean_q: 0.4318
Epoch 323/323
 - 0s - loss: 0.5283 - mean_q: 0.7687
Epoch 324/324
 - 0s - loss: 0.3236 - mean_q: 1.0043
Epoch 325/325
 - 0s - loss: 0.4947 - mean_q: 1.0299
Epoch 326/326
 - 0s - loss: 0.5144 - mean_q: 0.6339
Epoch 327/327
 - 0s - loss: 0.6761 - mean_q: 0.6760
Epoch 328/328
 - 0s - loss: 0.2320 - mean_q: 0.8825
Epoch 329/329
 - 0s - loss: 1.0412 - mean_q: 0.9014
Epoch 330/330
 - 0s - loss: 0.4563 - mean_q: 1.0429
Epoch 331/331
 - 0s - loss: 0.7376 - mean_q: 0.8308
Epoch 332/332
 - 0s - loss: 0.5928 - mean_q: 0.5377
Epoch 333/33

Epoch 472/472
 - 0s - loss: 0.3967 - mean_q: 0.6732
Epoch 473/473
 - 0s - loss: 0.5878 - mean_q: 0.6825
Epoch 474/474
 - 0s - loss: 0.7848 - mean_q: 0.6226
Epoch 475/475
 - 0s - loss: 0.2923 - mean_q: 0.5491
Epoch 476/476
 - 0s - loss: 0.4468 - mean_q: 0.5384
Epoch 477/477
 - 0s - loss: 0.5841 - mean_q: 0.5617
Epoch 478/478
 - 0s - loss: 0.3565 - mean_q: 0.5928
Epoch 479/479
 - 0s - loss: 0.4940 - mean_q: 0.6232
Epoch 480/480
 - 0s - loss: 0.5426 - mean_q: 0.6750
Epoch 481/481
 - 0s - loss: 0.5157 - mean_q: 0.7951
Epoch 482/482
 - 0s - loss: 0.6681 - mean_q: 1.0068
Epoch 483/483
 - 0s - loss: 0.1945 - mean_q: 0.8908
Epoch 484/484
 - 0s - loss: 0.5966 - mean_q: 0.9586
Epoch 485/485
 - 0s - loss: 1.0213 - mean_q: 1.1629
Epoch 486/486
 - 0s - loss: 0.4755 - mean_q: 0.9331
Epoch 487/487
 - 0s - loss: 0.4117 - mean_q: 0.8057
Epoch 488/488
 - 0s - loss: 0.8378 - mean_q: 0.7669
Epoch 489/489
 - 0s - loss: 0.3646 - mean_q: 0.6847
Epoch 490/490
 - 0s - loss: 0.3932 - mean_q: 0.6680
Epoch 491/49

Epoch 630/630
 - 0s - loss: 0.4977 - mean_q: 1.0759
Epoch 631/631
 - 0s - loss: 0.2601 - mean_q: 1.0429
Epoch 632/632
 - 0s - loss: 0.0218 - mean_q: 0.9980
Epoch 633/633
 - 0s - loss: 0.3380 - mean_q: 0.9466
Epoch 634/634
 - 0s - loss: 0.7377 - mean_q: 0.9387
Epoch 635/635
 - 0s - loss: 0.4198 - mean_q: 0.9532
Epoch 636/636
 - 0s - loss: 0.4988 - mean_q: 0.9999
Epoch 637/637
 - 0s - loss: 0.1783 - mean_q: 0.9707
Epoch 638/638
 - 0s - loss: 0.4179 - mean_q: 0.9396
Epoch 639/639
 - 0s - loss: 0.4954 - mean_q: 0.9483
Epoch 640/640
 - 0s - loss: 0.3391 - mean_q: 0.9417
Epoch 641/641
 - 0s - loss: 0.6627 - mean_q: 0.9154
Epoch 642/642
 - 0s - loss: 0.7405 - mean_q: 0.9284
Epoch 643/643
 - 0s - loss: 0.3370 - mean_q: 0.9380
Epoch 644/644
 - 0s - loss: 0.6624 - mean_q: 0.9895
Epoch 645/645
 - 0s - loss: 0.7328 - mean_q: 0.9952
Epoch 646/646
 - 0s - loss: 0.3379 - mean_q: 0.9907
Epoch 647/647
 - 0s - loss: 0.4977 - mean_q: 0.9995
Epoch 648/648
 - 0s - loss: 0.4997 - mean_q: 0.9856
Epoch 649/64

...

Epoch 36647/36647
 - 0s - loss: 0.5706 - mean_q: 3.4711
Epoch 36648/36648
 - 0s - loss: 0.6510 - mean_q: 3.4673
Epoch 36649/36649
 - 0s - loss: 0.4214 - mean_q: 3.4665
Epoch 36650/36650
 - 0s - loss: 0.5781 - mean_q: 3.4660
Epoch 36651/36651
 - 0s - loss: 0.5894 - mean_q: 3.4665
Epoch 36652/36652
 - 0s - loss: 0.1935 - mean_q: 3.4648
Epoch 36653/36653
 - 0s - loss: 0.5728 - mean_q: 3.4610
Epoch 36654/36654
 - 0s - loss: 0.5721 - mean_q: 3.4579
Epoch 36655/36655
 - 0s - loss: 34.6021 - mean_q: 3.4637
Epoch 36656/36656
 - 0s - loss: 0.2659 - mean_q: 3.4915
Epoch 36657/36657
 - 0s - loss: 0.4942 - mean_q: 3.5073
Epoch 36658/36658
 - 0s - loss: 0.6556 - mean_q: 3.5160
Epoch 36659/36659
 - 0s - loss: 35.1843 - mean_q: 3.5232
Epoch 36660/36660
 - 0s - loss: 0.4897 - mean_q: 3.5372
Epoch 36661/36661
 - 0s - loss: 0.5731 - mean_q: 3.5489
Epoch 36662/36662
 - 0s - loss: 0.6387 - mean_q: 3.5566
Epoch 36663/36663
 - 0s - loss: 0.6432 - mean_q: 3.5641
Epoch 36664/36664
 - 0s - loss: 0.4987 - mean_

Epoch 36794/36794
 - 0s - loss: 0.7953 - mean_q: 3.6383
Epoch 36795/36795
 - 0s - loss: 0.4265 - mean_q: 3.6513
Epoch 36796/36796
 - 0s - loss: 0.3444 - mean_q: 3.6568
Epoch 36797/36797
 - 0s - loss: 0.5801 - mean_q: 3.6591
Epoch 36798/36798
 - 0s - loss: 0.5059 - mean_q: 3.6587
Epoch 36799/36799
 - 0s - loss: 34.8244 - mean_q: 3.6550
Epoch 36800/36800
 - 0s - loss: 0.8709 - mean_q: 3.6555
Epoch 36801/36801
 - 0s - loss: 0.6509 - mean_q: 3.6622
Epoch 36802/36802
 - 0s - loss: 0.6403 - mean_q: 3.6653
Epoch 36803/36803
 - 0s - loss: 35.0578 - mean_q: 3.6673
Epoch 36804/36804
 - 0s - loss: 0.6600 - mean_q: 3.6732
Epoch 36805/36805
 - 0s - loss: 0.2806 - mean_q: 3.6783
Epoch 36806/36806
 - 0s - loss: 0.1217 - mean_q: 3.6760
Epoch 36807/36807
 - 0s - loss: 0.4170 - mean_q: 3.6690
Epoch 36808/36808
 - 0s - loss: 0.3642 - mean_q: 3.6615
Epoch 36809/36809
 - 0s - loss: 0.5780 - mean_q: 3.6544
Epoch 36810/36810
 - 0s - loss: 0.5787 - mean_q: 3.6482
Epoch 36811/36811
 - 0s - loss: 0.5827 - mean_

Epoch 36940/36940
 - 0s - loss: 0.6442 - mean_q: 3.6626
Epoch 36941/36941
 - 0s - loss: 0.7332 - mean_q: 3.6569
Epoch 36942/36942
 - 0s - loss: 0.5091 - mean_q: 3.6522
Epoch 36943/36943
 - 0s - loss: 0.7271 - mean_q: 3.6485
Epoch 36944/36944
 - 0s - loss: 0.5745 - mean_q: 3.6466
Epoch 36945/36945
 - 0s - loss: 2.8469 - mean_q: 3.6456
Epoch 36946/36946
 - 0s - loss: 0.5787 - mean_q: 3.6453
Epoch 36947/36947
 - 0s - loss: 0.4337 - mean_q: 3.6427
Epoch 36948/36948
 - 0s - loss: 0.4962 - mean_q: 3.6381
Epoch 36949/36949
 - 0s - loss: 0.5035 - mean_q: 3.6331
Epoch 36950/36950
 - 0s - loss: 0.3521 - mean_q: 3.6270
Epoch 36951/36951
 - 0s - loss: 0.1900 - mean_q: 3.6208
Epoch 36952/36952
 - 0s - loss: 0.2045 - mean_q: 3.6141
Epoch 36953/36953
 - 0s - loss: 0.5002 - mean_q: 3.6057
Epoch 36954/36954
 - 0s - loss: 0.2787 - mean_q: 3.5987
Epoch 36955/36955
 - 0s - loss: 0.5838 - mean_q: 3.5903
Epoch 36956/36956
 - 0s - loss: 2.8958 - mean_q: 3.5865
Epoch 36957/36957
 - 0s - loss: 0.5730 - mean_q:

Epoch 37087/37087
 - 0s - loss: 0.2723 - mean_q: 3.4413
Epoch 37088/37088
 - 0s - loss: 0.5672 - mean_q: 3.4325
Epoch 37089/37089
 - 0s - loss: 0.5689 - mean_q: 3.4258
Epoch 37090/37090
 - 0s - loss: 0.6586 - mean_q: 3.4205
Epoch 37091/37091
 - 0s - loss: 0.3545 - mean_q: 3.4169
Epoch 37092/37092
 - 0s - loss: 0.5055 - mean_q: 3.4111
Epoch 37093/37093
 - 0s - loss: 0.4984 - mean_q: 3.4062
Epoch 37094/37094
 - 0s - loss: 0.5019 - mean_q: 3.4023
Epoch 37095/37095
 - 0s - loss: 0.5776 - mean_q: 3.3995
Epoch 37096/37096
 - 0s - loss: 0.8825 - mean_q: 3.3982
Epoch 37097/37097
 - 0s - loss: 0.3539 - mean_q: 3.4002
Epoch 37098/37098
 - 0s - loss: 0.5052 - mean_q: 3.3996
Epoch 37099/37099
 - 0s - loss: 0.5705 - mean_q: 3.3985
Epoch 37100/37100
 - 0s - loss: 0.6660 - mean_q: 3.3974
Epoch 37101/37101
 - 0s - loss: 0.4197 - mean_q: 3.3954
Epoch 37102/37102
 - 0s - loss: 0.5737 - mean_q: 3.3943
Epoch 37103/37103
 - 0s - loss: 0.4954 - mean_q: 3.3940
Epoch 37104/37104
 - 0s - loss: 0.5827 - mean_q:

KeyboardInterrupt: 

Save the last weights

In [358]:
online_network.save_weights(os.path.join(weights_folder, 'weights_last.h5f'))

In [359]:
# Dump all scores to txt-file
with open(os.path.join(name, 'episode_scores.txt'), 'w') as file:
    for item in episode_scores:
        file.write("{}\n".format(item))

print(episode_scores)

[160.0, 150.0, 250.0, 150.0, 140.0, 170.0, 150.0, 210.0, 150.0, 120.0, 180.0, 230.0, 170.0, 210.0, 200.0, 290.0, 310.0, 250.0, 190.0, 240.0, 290.0, 360.0, 160.0, 150.0, 100.0, 320.0, 210.0, 170.0, 190.0, 160.0, 150.0, 180.0, 110.0, 210.0, 220.0, 200.0, 240.0, 230.0, 180.0, 150.0, 270.0, 190.0, 120.0, 250.0, 70.0, 210.0, 220.0, 310.0, 220.0, 120.0, 630.0, 230.0, 250.0, 150.0, 410.0, 150.0, 210.0, 190.0, 290.0, 210.0, 270.0, 230.0, 190.0, 430.0, 180.0, 250.0, 170.0, 240.0, 130.0, 180.0, 180.0, 220.0, 250.0, 320.0, 200.0, 210.0, 190.0, 370.0, 160.0, 240.0, 180.0, 220.0, 170.0, 200.0, 240.0, 170.0, 180.0, 140.0, 260.0, 280.0, 170.0, 330.0, 230.0, 180.0, 150.0, 450.0, 270.0, 160.0, 190.0, 200.0, 250.0, 200.0, 210.0, 170.0, 240.0, 130.0, 210.0, 240.0, 170.0, 140.0, 210.0, 170.0, 190.0, 200.0, 320.0, 230.0, 180.0, 220.0, 190.0, 210.0, 440.0, 310.0, 180.0, 210.0, 280.0, 410.0, 250.0, 160.0, 190.0, 200.0, 310.0, 270.0, 220.0, 230.0, 190.0, 150.0, 470.0, 280.0, 250.0, 260.0, 700.0, 170.0, 210.0,

### Test the DQN model

In [360]:
def test_dqn(env, n_games, model, nb_actions, skip_start, eps=0.01, render=False, sleep_time=0.01):
    scores = []
    for i in range(n_games):
        obs = env.reset()
        score = 0
        done = False
        for skip in range(skip_start):  # skip the start of each game (it's just freezing time before game starts)
            obs, reward, done, info = env.step(0)
            score += reward
        while not done:
            state = obs
            q_values = model.predict(np.array([state]))[0]
            action = epsilon_greedy(q_values, eps, nb_actions)
            obs, reward, done, info = env.step(action)
            score += reward
            if render:
                env.render()
                time.sleep(sleep_time)
                if done:
                    time.sleep(1)
        scores.append(score)
    return scores

In [361]:
ngames = 100
scores = test_dqn(env, ngames, online_network, nb_actions, skip_start, render=False)
print(scores)

[220.0, 210.0, 300.0, 340.0, 220.0, 210.0, 210.0, 320.0, 320.0, 380.0, 330.0, 210.0, 230.0, 330.0, 220.0, 210.0, 220.0, 220.0, 320.0, 220.0, 220.0, 220.0, 220.0, 220.0, 990.0, 340.0, 210.0, 220.0, 210.0, 210.0, 210.0, 500.0, 210.0, 210.0, 220.0, 340.0, 220.0, 370.0, 210.0, 210.0, 210.0, 220.0, 210.0, 210.0, 220.0, 220.0, 220.0, 220.0, 220.0, 310.0, 360.0, 210.0, 210.0, 220.0, 480.0, 220.0, 250.0, 410.0, 320.0, 220.0, 210.0, 320.0, 220.0, 210.0, 220.0, 330.0, 220.0, 250.0, 220.0, 210.0, 220.0, 210.0, 220.0, 220.0, 220.0, 220.0, 400.0, 330.0, 970.0, 210.0, 210.0, 220.0, 320.0, 220.0, 220.0, 210.0, 220.0, 210.0, 200.0, 230.0, 220.0, 210.0, 220.0, 210.0, 220.0, 220.0, 220.0, 240.0, 210.0, 320.0]


In [362]:
sum(scores)/100

263.4

In [363]:
print('\nFifth percentile: ',np.percentile(scores,5))


Fifth percentile:  210.0


In [364]:
print([ np.percentile(scores, p) for p in [0, 25, 50, 75, 100] ])

[200.0, 210.0, 220.0, 262.5, 990.0]
